In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
# Rename pictures so we can trace them better after classification
dir_list = ['data/test/shoe', 'data/test/no_shoe']

for dir in dir_list:
    i = 1
    try:
        for filename in os.listdir(dir): 
            if filename == 'Thumbs.db':
                continue
            dst = 'test_picture' + '_' + str(i) + ".jpg"
            src = dir + '/' + filename 
            dst = dir + '/' + dst 

            # rename() function will 
            # rename all the files 
            os.rename(src, dst) 
            i += 1
    except Exception as e:
        print(e)
    finally:
        i += 1

        print("formatted " + str(dir))

formatted data/test/shoe
formatted data/test/no_shoe


In [3]:
# Load model
model = load_model('models/first_model/model.h5')

In [4]:
# Create generator to pull pictures from directory
datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_directory(
        'data/test',
        target_size=(300, 300),
        batch_size=10,
        shuffle=False)

probabilities= model.predict_generator(generator, 50,  verbose=1)

Found 176 images belonging to 2 classes.


C:\Users\Lucas\Anaconda3\envs\TFEnv\lib\site-packages\PIL\Image.py:966: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


50/50 [==============================] - 68s 1s/step


In [5]:
# Zip predictions to filenames
mapped_preds = np.array(list(zip(probabilities.flatten(),np.asarray(generator.filenames))))
mapped_preds_list = mapped_preds.tolist()
# mapped_preds_list

In [6]:
# Pack predictions, correct label, file name and predicted probability in a list
def map_correct(prediction_touple):
    if float(prediction_touple[0]) < 0.5:
        prediction = 0
    else:
        prediction = 1
    
    if prediction_touple[1][0] == 's':
        correct_pred = 1
    else:
        correct_pred = 0
    
    return list([prediction, correct_pred, prediction_touple[1], prediction_touple[0]])

In [7]:
# Reformat to dataframe
list_for_df = list(map(map_correct, mapped_preds_list))
evaluation_df = pd.DataFrame(list_for_df, columns = ['prediction' , 'correct label', 'file name', 'predicted probability']) 
# evaluation_df

In [8]:
# Classification report
print(classification_report(evaluation_df['correct label'], evaluation_df['prediction']))

              precision    recall  f1-score   support

           0       0.89      0.87      0.88        39
           1       0.96      0.97      0.97       137

   micro avg       0.95      0.95      0.95       176
   macro avg       0.93      0.92      0.93       176
weighted avg       0.95      0.95      0.95       176



In [9]:
# Print complete df
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(evaluation_df)

     prediction  correct label                    file name  \
0             0              0   no_shoe\test_picture_1.jpg   
1             0              0  no_shoe\test_picture_10.jpg   
2             0              0  no_shoe\test_picture_11.jpg   
3             0              0  no_shoe\test_picture_12.jpg   
4             0              0  no_shoe\test_picture_13.jpg   
5             0              0  no_shoe\test_picture_14.jpg   
6             0              0  no_shoe\test_picture_15.jpg   
7             0              0  no_shoe\test_picture_16.jpg   
8             1              0  no_shoe\test_picture_17.jpg   
9             1              0  no_shoe\test_picture_18.jpg   
10            1              0  no_shoe\test_picture_19.jpg   
11            0              0   no_shoe\test_picture_2.jpg   
12            0              0  no_shoe\test_picture_20.jpg   
13            0              0  no_shoe\test_picture_21.jpg   
14            0              0  no_shoe\test_picture_22

In [10]:
# Print confusion matrix
confusion_matrix(evaluation_df['correct label'], evaluation_df['prediction'])

array([[ 34,   5],
       [  4, 133]], dtype=int64)